# Simon's algorithm

Simon's algorithm is closely related in principle to the Bernstein-Vazirani algorithm and the Deutsch-Josza algorithm, (which are also explained in other notebooks in this repository).

We are given an "oracle" and promised that its outputs are either **one-to-one** or **two-to-one**, i.e., it either has **exactly** as many possible outputs as possible inputs or **exactly half** as many possible outputs as possible inputs. In effect, all two-to-one functions can be thought of as being based on a "**hidden string**" encoded into the oracle, whereas one-to-one functions are the only case where the hidden string is "**0**." Our problem is therefore to determine the "hidden string" encoded into the oracle.

A classical computer, on average, needs to check about half the possible inputs to answer the question of what the hidden string is; a quantum computer needs **exponentially fewer** queries of the oracle to determine the same answer.

(...So the conventional disciplinary thinking goes, at least. But notice that we are doing 120 qubits of Simon's algorithm below, and we could argue we only need a single query, with PyQrack!)

In [1]:
import os
import random
from collections import Counter
from pyqrack import QrackSimulator

# Tell Qrack that it can use as many stabilizer qubits as it wants,
# without worrying about converting to ket for GPU:
os.environ['QRACK_MAX_CPU_QB']='-1'

# Range is up to 2 ** input_size - 1
input_size = 60

# Arbitrary value within range:
hidden_bits = 2**40 - 1285

num_qubits = input_size * 2
oracle_qubits = [*range(input_size)]

oracle_swap_tuples = []
for i in range(input_size // 2):
    i = random.randint(input_size, (input_size * 2) - 1)
    j = random.randint(input_size, (input_size * 2) - 1)
    if i != j:
        oracle_swap_tuples.append((i, j))

You may change the `hidden_bits` above to be any value that can fit within the oracle register width. Your `hidden_bits` corresponds to the oracle below:

In [2]:
def oracle(sim):
    # The first portion of this oracle is effectively a one-to-one oracle,
    # in itself, (though not the only possible one-to-one oracle).
    for i in oracle_qubits:
        sim.mcx([i], i + input_size)

    # The second portion of this oracle makes it two-to-one,
    # (unless "hidden_bits" is 0):
    for i in oracle_qubits:
        # We pick any one of the "set" hidden bits:
        if (hidden_bits >> i) & 1:
            for j in oracle_qubits:
                # Act CNOT between the "set" hidden bit in input and all hidden bits in output:
                if (hidden_bits >> j) & 1:
                    sim.mcx([i], j + input_size)
            # One bit in the "i" loop is sufficient:
            break;

    # We can now swap at random in the output register
    # and still satisfy our promises about the oracle:
    for i in oracle_swap_tuples:
         sim.swap(i[0], i[1])

This is the algorithm itself. (A classical simulator of quantum computers can run the "unitary preamble" exactly once and then secondarily "sample" the measurement distribution of the end state.)

In [3]:
# Prepare the initial register state:
sim = QrackSimulator(num_qubits, isCpuGpuHybrid=False, isOpenCL=False)
for i in oracle_qubits:
    sim.h(i)

# Make exactly one query to the oracle:
oracle(sim)

# Finish the unitary portion of the algorithm, with the result from the oracle...
# (It is often presented that measurement of the second register needs to occur before this step;
# this is actually wholly theoretically unnecessary in the ideal, due to locality of quantum information.)
for i in oracle_qubits:
    sim.h(i)

# The cost in "shot" count is linear, at this point.
# results = Counter(sim.measure_shots(oracle_qubits, 32768))

# The shared library API definition prevents high-width sampling, above a limit.
results = {}
for i in range(32768):
    s = QrackSimulator(cloneSid = sim.sid)
    s.m_all()
    result = 0
    for j in oracle_qubits:
        if s.m(j):
            result |= 1 << j
    if result in results:
        results[result] = results[result] + 1
    else:
        results[result] = 1

The ultimate point of the algorithm is that, by solving the system of equations below, **where the value of `hidden_string` is an algebraic unknown**, we can determine the value of `hidden_string`, with a probabilistic likelihood that converges exponentially faster than a "classical" algorithm, in the number of measurement "shots." (In a real hardware quantum computer, the number of "shots" would correspond to exactly the number of queries of the oracle).

Let's check if our implied system of equations is right:

In [4]:
for key in results.keys():
    x = (bin(hidden_bits & key).count("1")) & 1
    if x > 0:
        print("Failed!", hidden_bits, ".", key, "=", x, "(mod 2)")
        break
    print(hidden_bits, ".", key, "=", x, "(mod 2)")

1099511626491 . 474242951795297641 = 0 (mod 2)
1099511626491 . 659099635130655281 = 0 (mod 2)
1099511626491 . 51178858570474602 = 0 (mod 2)
1099511626491 . 635728698779242709 = 0 (mod 2)
1099511626491 . 823279085390939289 = 0 (mod 2)
1099511626491 . 1041503302952845821 = 0 (mod 2)
1099511626491 . 391348871994384399 = 0 (mod 2)
1099511626491 . 919881780298450263 = 0 (mod 2)
1099511626491 . 319503067944372138 = 0 (mod 2)
1099511626491 . 920440657193389723 = 0 (mod 2)
1099511626491 . 950619112472199833 = 0 (mod 2)
1099511626491 . 1024804579283730778 = 0 (mod 2)
1099511626491 . 345636513994835799 = 0 (mod 2)
1099511626491 . 136654669094625280 = 0 (mod 2)
1099511626491 . 695851980183806778 = 0 (mod 2)
1099511626491 . 1149326575842590008 = 0 (mod 2)
1099511626491 . 1006970017584122686 = 0 (mod 2)
1099511626491 . 53403116339061214 = 0 (mod 2)
1099511626491 . 991056449999313329 = 0 (mod 2)
1099511626491 . 575034400036708562 = 0 (mod 2)
1099511626491 . 73808448631281434 = 0 (mod 2)
109951162649

1099511626491 . 1105298974797163964 = 0 (mod 2)
1099511626491 . 345150063133653586 = 0 (mod 2)
1099511626491 . 656250522768110589 = 0 (mod 2)
1099511626491 . 23520726569698949 = 0 (mod 2)
1099511626491 . 719440128585462332 = 0 (mod 2)
1099511626491 . 691124834902359053 = 0 (mod 2)
1099511626491 . 920908657736646717 = 0 (mod 2)
1099511626491 . 1065569807425773147 = 0 (mod 2)
1099511626491 . 680166776724246738 = 0 (mod 2)
1099511626491 . 28027866467060847 = 0 (mod 2)
1099511626491 . 533183784653622990 = 0 (mod 2)
1099511626491 . 324138656392499024 = 0 (mod 2)
1099511626491 . 617563165961607656 = 0 (mod 2)
1099511626491 . 382340940379483692 = 0 (mod 2)
1099511626491 . 9598128754453534 = 0 (mod 2)
1099511626491 . 935406305509974757 = 0 (mod 2)
1099511626491 . 570343058488734711 = 0 (mod 2)
1099511626491 . 273722299935207946 = 0 (mod 2)
1099511626491 . 1090327456955847520 = 0 (mod 2)
1099511626491 . 720152155908145539 = 0 (mod 2)
1099511626491 . 235307325436144828 = 0 (mod 2)
1099511626491 